In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from pathlib import Path
from os import chdir
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.estimators import ASmoothMapEstimator, TSMapEstimator
from gammapy.estimators.utils import find_peaks
from gammapy.datasets import MapDataset
from gammapy.modeling.models import (
    Models,
    SkyModel,
    PowerLawSpectralModel,
    PointSpatialModel,
    DiskSpatialModel,
)
from gammapy.analysis import Analysis, AnalysisConfig
from gammapy.irf import PSFMap, EDispKernelMap
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
from gammapy.data import DataStore
from gammapy.datasets import MapDatasetEventSampler, MapDataset
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.utils.scripts import read_yaml

In [2]:
config = AnalysisConfig()
# The config file is now empty, with only a few defaults specified.
print(config)

AnalysisConfig

    general:
        log: {level: info, filename: null, filemode: null, format: null, datefmt: null}
        outdir: .
        n_jobs: 1
        datasets_file: null
        models_file: null
    observations:
        datastore: $GAMMAPY_DATA/hess-dl3-dr1
        obs_ids: []
        obs_file: null
        obs_cone: {frame: null, lon: null, lat: null, radius: null}
        obs_time: {start: null, stop: null}
        required_irf: [aeff, edisp, psf, bkg]
    datasets:
        type: 1d
        stack: true
        geom:
            wcs:
                skydir: {frame: null, lon: null, lat: null}
                binsize: 0.02 deg
                width: {width: 5.0 deg, height: 5.0 deg}
                binsize_irf: 0.2 deg
            selection: {offset_max: 2.5 deg}
            axes:
                energy: {min: 1.0 TeV, max: 10.0 TeV, nbins: 5}
                energy_true: {min: 0.5 TeV, max: 20.0 TeV, nbins: 16}
        map_selection: [counts, exposure, background, psf, ed

In [3]:
# Selecting the observations
config.observations.datastore = ("../")
# We define the cone search parameters
config.observations.obs_cone.frame = "galactic"
config.observations.obs_cone.lon = "72 deg"
config.observations.obs_cone.lat = "1.5 deg"
config.observations.obs_cone.radius = "15 deg"

In [4]:
# We want to perform a 3D analysis
config.datasets.type = "3d"
# We want to stack the data into a single reduced dataset
config.datasets.stack = True

# We fix the WCS geometry of the datasets
config.datasets.geom.wcs.skydir = {
    "lon": "72 deg",
    "lat": "1.5 deg",
    "frame": "galactic",
}
config.datasets.geom.wcs.width = {"width": "22 deg", "height": "12 deg"}
config.datasets.geom.wcs.binsize = "0.02 deg"

# We now fix the energy axis for the counts map
config.datasets.geom.axes.energy.min = "1 TeV"
config.datasets.geom.axes.energy.max = "10 TeV"
config.datasets.geom.axes.energy.nbins = 10

# We now fix the energy axis for the IRF maps (exposure, etc)
config.datasets.geom.axes.energy_true.min = "0.5 TeV"
config.datasets.geom.axes.energy_true.max = "20 TeV"
config.datasets.geom.axes.energy_true.nbins = 20

config.datasets.background.method = "fov_background"
config.datasets.background.parameters = {"method": "scale"}

config.fit.fit_range.min = 1 * u.TeV
config.fit.fit_range.max = 10 * u.TeV
config.flux_points.energy = {"min": "1 TeV", "max": "10 TeV", "nbins": 4}
config.excess_map.correlation_radius = 0.1 * u.deg

In [5]:
analysis = Analysis(config)

Setting logging config: {'level': 'INFO', 'filename': None, 'filemode': None, 'format': None, 'datefmt': None}


In [6]:
analysis.get_observations()
analysis.get_datasets()

Fetching observations.
Observations selected: 180 out of 180.
Number of selected observations: 180
Creating reference dataset and makers.
Creating the background Maker.
Start the data reduction loop.
Computing dataset for observation 3
Running MapDatasetMaker
Running SafeMaskMaker
No default upper safe energy threshold defined for obs 3
No default lower safe energy threshold defined for obs 3
Running FoVBackgroundMaker
Computing dataset for observation 31
Running MapDatasetMaker
Running SafeMaskMaker
No default upper safe energy threshold defined for obs 31
No default lower safe energy threshold defined for obs 31
Running FoVBackgroundMaker
Computing dataset for observation 124
Running MapDatasetMaker
Running SafeMaskMaker
No default upper safe energy threshold defined for obs 124
No default lower safe energy threshold defined for obs 124
Running FoVBackgroundMaker
Computing dataset for observation 15
Running MapDatasetMaker
Running SafeMaskMaker
No default upper safe energy threshold 

In [35]:
print(Path.cwd())
chdir("..")
print(Path.cwd())

filename = "skymodel/global_skymodel.yaml"
models_read = Models.read(filename)
models_read.remove("diffuse")

/home/antonio/Analysis/GAMMAPY/GammaCygni/analysis
/home/antonio/Analysis/GAMMAPY/GammaCygni


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.


In [39]:
analysis.set_models(models_read, extend=False)

Reading model.
Models

Component 0: SkyModel

  Name                      : PSR J1958+2846
  Datasets names            : None
  Spectral model type       : LogParabolaSpectralModel
  Spatial  model type       : GaussianSpatialModel
  Temporal model type       : 
  Parameters:
    amplitude                     :   3.00e-15   +/- 0.0e+00 1 / (cm2 s TeV)
    reference             (frozen):      7.000       TeV         
    alpha                         :      2.200   +/-    0.00             
    beta                          :      0.045   +/-    0.00             
    lon_0                         :    299.667   +/-    0.00 deg         
    lat_0                         :     28.765   +/-    0.00 deg         
    sigma                         :      0.150   +/-    0.00 deg         
    e                     (frozen):      0.000                   
    phi                   (frozen):      0.000       deg         

Component 1: SkyModel

  Name                      : PSR J1954+2836
  Dataset

In [18]:
chdir("analysis")
print(Path.cwd())

/home/antonio/Analysis/GAMMAPY/GammaCygni/analysis
